<a href="https://colab.research.google.com/github/Munya07/NLP/blob/master/3_Chat_bot/Deep_pavlov/Chatbot_deeppavlov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q deeppavlov

     |████████████████████████████████| 675kB 4.9MB/s 
     |████████████████████████████████| 31.2MB 1.3MB/s 
     |████████████████████████████████| 12.2MB 36.2MB/s 
     |████████████████████████████████| 61kB 22.0MB/s 
     |████████████████████████████████| 3.4MB 42.4MB/s 
     |████████████████████████████████| 92kB 26.2MB/s 
     |████████████████████████████████| 12.4MB 40.5MB/s 
     |████████████████████████████████| 11.8MB 35.6MB/s 
     |████████████████████████████████| 8.0MB 2.1MB/s 
     |████████████████████████████████| 4.1MB 40.7MB/s 
     |████████████████████████████████| 92kB 27.6MB/s 
     |████████████████████████████████| 307kB 45.1MB/s 
     |████████████████████████████████| 61kB 23.2MB/s 
     |████████████████████████████████| 51kB 19.7MB/s 
     |████████████████████████████████| 51kB 19.8MB/s 
     |████████████████████████████████| 143kB 48.7MB/s 
     |████████████████████████████████| 61kB 20.7MB/s 
     |████████████████████████████████| 51kB 20.0MB/s 

In [0]:
!pip install geopy
!pip install python-forecastio
!pip install darkskylib
!pip install dateparser

In [0]:
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from geopy.geocoders import Nominatim


2019-06-05 21:13:37.948 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/vectorizer/tfidf_vectorizer_ruwiki.pkl to /content/My drive/colab/vectorizer/tfidf_vectorizer_ruwiki.pkl
100%|██████████| 140M/140M [00:25<00:00, 5.49MB/s]
2019-06-05 21:14:04.256 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/faq/school/tfidf_cos_sim_classifier.pkl to /content/My drive/colab/faq/tfidf_cos_sim_classifier.pkl
100%|██████████| 12.0k/12.0k [00:00<00:00, 3.88MB/s]
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unz

In [0]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill
hello = PatternMatchingSkill(responses=['Hi, how can l help you'], patterns=['hi', 'hello', 'How are you'], default_confidence=0.3)
bye = PatternMatchingSkill(responses=['Good bye'], patterns=['bye bye', 'bye'], default_confidence=0.3)
weather = PatternMatchingSkill(responses=['For which location and date, l can only forecast for 1 week'], patterns=['weather', 'rain','temperature'], default_confidence=0.3)
check=PatternMatchingSkill(responses=['Let me check'], patterns=[''], default_confidence=0.2)

In [0]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector
agent = DefaultAgent([hello, bye, weather,check], skills_selector=HighestConfidenceSelector())

In [0]:

  
def run(input): 
  doc = nlp(input)
  dct = dict(( X.label_,X.text) for X in doc.ents)
  location = dct.get('GPE')
  date=dct.get('DATE')
  print(date, location)
    
    
  if location!=None and date!=None:

    
    import dateparser
    d=dateparser.parse(date)
    d=str(d)
    d=d[:10].replace('-', '/')
    print(d)

    from darksky import forecast
    from datetime import date, timedelta

    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="specify_your_app_name_here")
    location_ = geolocator.geocode(location)

    BOSTON = location_.latitude, location_.longitude

    weekday = date.today()
    with forecast('d6c7239325b1a5e9543d57c7cd385ff4', *BOSTON) as boston:
      print(boston.daily.summary, end='\n---\n')
      for day in boston.daily:
          day = dict(day = date.strftime(weekday, '%x'),
                     sum = day.summary,
                     tempMin = day.temperatureMin,
                     tempMax = day.temperatureMax)
          l = day.get('day')           
          if l==d:
            print(response='{day}: {sum} Temp range: {tempMin} - {tempMax}'.format(**day))
          weekday += timedelta(days=1)
      
    
  else:
    if location==None and date!=None:
      print('sorry your location might be wrong')
      
    elif date==None and location!=None:
      print('sorry your  date might be wrong, l can only give you weather in the next 7 days')
    else:
      print('sorry your location and date might be wrong, l can only give you weather in the next 7 days')



In [0]:

while x==0:
  test_text = input ("Chat with weather chatbot : ")
  print(agent([test_text]))
  a=str(agent([test_text])[0])
  if a=='Let me check':
    run(test_text)
 


Chat with weather chatbot : hie
['Hi, how can l help you']
Chat with weather chatbot : how are you
['Hi, how can l help you']
Chat with weather chatbot : whats the weather in India
['For which location and date, l can only forecast for 1 week']
Chat with weather chatbot : India tomorrow
['Let me check']
tomorrow India
2019/06/07
No precipitation throughout the week, with high temperatures falling to 102°F on Tuesday.
---


KeyboardInterrupt: ignored